In [1]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
import math
import scipy as sp
from scipy.interpolate import interp1d, splrep, UnivariateSpline
from astropy.io import fits
from tqdm import tqdm

In [3]:
"""This function reads a very specific format of .txt file, where the first three lines 
contain information but no data. The data is then contained in 7 columns, 181 rows."""
def ascii_txt_reader(filepath, data_start, header_check):    
    with open(filepath, "r") as file:
        lines = file.readlines()
        if header_check == True:
            print(lines[0], lines[1], lines[2])
        line_useful = lines[data_start:]

        whole_data = []
        for line in line_useful: #for each line of data 
            python_values = line.replace('D', 'E') #converts each line to python sci. notation
            stripped_values = python_values.strip()
            row = stripped_values.split() #creates rows by splitting into lines
            whole_data.append(row) #appending each line to an array
        #to extract rows, don't transpose in the following line:
        cols_str = np.transpose(np.array(whole_data)) #transposes array from rows to columns
        cols = cols_str.astype(float) #takes the columns and converts all the values from strings to floats
        return cols

In [3]:
def write_to_file(sil_file, car_file, w1, w2, new_txt_file, Q, phase_fxn, angle_array):
    sil_cols = ascii_txt_reader(sil_file,3, False)
    car_cols = ascii_txt_reader(car_file,3, False)

    if Q == True:
        Q_phase_fxn_sil = (sil_cols[1] - sil_cols[2]) * 0.5
        Q_phase_fxn_car = (car_cols[1] - car_cols[2]) * 0.5
        f1 = Q_phase_fxn_sil
        f2 = Q_phase_fxn_car
        weighted_Q = (w1*f1 + w2*f2)/(w1+w2)
        f = open(new_txt_file, "w")
        for value in weighted_Q:
            f.write(str(value))
            f.write("\n")
        f.close()
        return new_txt_file
    if phase_fxn == True:
        phase_fxn_sil = sil_cols[5]
        phase_fxn_car = car_cols[5]
        f1 = phase_fxn_sil
        f2 = phase_fxn_car
        weighted_phase_fxn = (w1*f1 + w2*f2)/(w1+w2)
        f = open(new_txt_file, "w")
        for value in weighted_phase_fxn:
            f.write(str(value)) #if i didn't have string here would it write as a float
            f.write("\n")
        f.close()
        return new_txt_file
    if angle_array == True:
        f = open(new_txt_file, "w")
        for value in sil_cols[0]:
            f.write(str(value)) #if i didn't have string here would it write as a float
            f.write("\n")
        f.close()
        return new_txt_file


In [14]:
def write_rayleigh():
    sil_cols = ascii_txt_reader("compaSilreddraine.txt",3, False)
    f = open('rayleigh_phase.txt', "w")
    phase_fxn = 3*((((np.cos(sil_cols[0]*np.pi/180.))**2)+1))/4
    #print(np.shape(phase_fxn))
    for value in phase_fxn:
        f.write(str(value)) #
        f.write("\n")
    f.close()
    f = open('rayleigh_pol.txt', "w")
    phase_fxn = 3*(((np.cos(sil_cols[0]*np.pi/180.))**2) - 1)/4
    #print(np.shape(phase_fxn))
    for value in phase_fxn:
        f.write(str(value)) 
        f.write("\n")
    f.close()

In [15]:
def write_rayleigh_g():
    sil_cols = ascii_txt_reader("compaSilgreendraine.txt",3, False)
    f = open('rayleigh_phase_g.txt', "w")
    phase_fxn = 3*((((np.cos(sil_cols[0]*np.pi/180.))**2)+1))/4
    #print(np.shape(phase_fxn))
    for value in phase_fxn:
        f.write(str(value)) #
        f.write("\n")
    f.close()
    f = open('rayleigh_pol_g.txt', "w")
    phase_fxn = 3*(((np.cos(sil_cols[0]*np.pi/180.))**2) - 1)/4
    #print(np.shape(phase_fxn))
    for value in phase_fxn:
        f.write(str(value)) 
        f.write("\n")
    f.close()

In [16]:
write_rayleigh_g()


In [17]:
write_rayleigh()

In [8]:
write_to_file("compaSilreddraine.txt", 'compLamCredzubko.txt', 133.197, 43.7558, "weighted_Q_0.txt", True, False, False)
write_to_file("compaSilreddraine.txt", 'compLamCredzubko.txt', 133.197, 43.7558, "weighted_phase_fxn_0.txt", False, True, False)
write_to_file("compaSilreddraine.txt", 'compLamCredzubko.txt', 133.197, 43.7558, "angle_array.txt", False, False, True)

write_to_file("compaSilgreendraine.txt", 'compLamCgreenzubko.txt', 174.598, 47.5795, "weighted_Q_1.txt", True, False, False)
write_to_file("compaSilgreendraine.txt", 'compLamCgreenzubko.txt', 174.598, 47.5795, "weighted_phase_fxn_1.txt", False, True, False)

write_to_file("wdaSil31reddraine.txt", 'wdGra31redzubko.txt', 2137.96, 708.008, "weighted_Q_2.txt", True, False, False)
write_to_file("wdaSil31reddraine.txt", 'wdGra31redzubko.txt', 2137.96, 708.008, "weighted_phase_fxn_2.txt", False, True, False)

write_to_file("wdaSil31greendraine.txt", 'wdGra31greenzubko.txt', 3254.56, 780.729, "weighted_Q_3.txt", True, False, False)
write_to_file("wdaSil31greendraine.txt", 'wdGra31greenzubko.txt', 3254.56, 780.729, "weighted_phase_fxn_3.txt", False, True, False)

write_to_file("wdaSil55reddraine.txt", 'wdGra55redzubko.txt', 2863.63, 451.709, "weighted_Q_4.txt", True, False, False)
write_to_file("wdaSil55reddraine.txt", 'wdGra55redzubko.txt', 2863.63, 451.709, "weighted_phase_fxn_4.txt", False, True, False)

write_to_file("wdaSil55greendraine.txt", 'wdGra55greenzubko.txt', 3645.40, 492.108, "weighted_Q_5.txt", True, False, False)
write_to_file("wdaSil55greendraine.txt", 'wdGra55greenzubko.txt', 3645.40, 492.108, "weighted_phase_fxn_5.txt", False, True, False)

#/fs/lustre/project/hp/pgmartin/Mie
#write_to_file("rayleigh.txt", False, False, False, True)
#write_rayleigh()

'weighted_phase_fxn_5.txt'

In [2]:
def weightg(g1,g2,w1,w2):
    weighted_g = (w1*g1 + w2*g2)/(w1+w2)
    print(weighted_g)

In [6]:
#DUSTEM red then green
#g:
# weightg(0.5807,0.4375,133.197, 43.7558)
# weightg(0.5854,0.5090,174.598, 47.5795)

# weightg(1.24,0.02998,133.197, 43.7558)
# weightg(1.183,0.0297,174.598, 47.5795)

ext1 = 155.121
sca1 = 133.197
Abs1 = 21.9236
ext2 = 118.658
sca2 = 43.7558
Abs2 = 74.9021

alb = (sca1+sca2)/(ext1+ext2)
alb2 = (sca1+sca2)/(sca1+sca2+Abs1+Abs2)
print(alb)
ext1 = 204.554
sca1 = 174.598

ext2 = 133.308
sca2 = 47.5795


alb = (sca1+sca2)/(ext1+ext2)
alb2 = (sca1+sca2)/(sca1+sca2+Abs1+Abs2)
print(alb)

0.6463344522406759
0.6575983685646803


In [10]:
#WD3.1 r then g
#g:
# weightg(0.5559, 0.4504, 2137.96, 708.008)
# weightg(0.5922, 0.5080, 3254.56, 780.729)
#albedo
# weightg(0.867116, 0.358898, 2137.96, 708.008)
# weightg(0.875130, 0.344685, 3254.56, 780.729)

# weightg(1.24,0.02998,133.197, 43.7558)
# weightg(1.183,0.0297,174.598, 47.5795)

sca1 = 2137.96
sca2 = 708.008
ext1 = 2465.60
ext2 = 1972.73
alb = (sca1+sca2)/(ext1+ext2)
print(alb)

sca1 = 3254.56
sca2 = 780.729
ext1 = 3718.94
ext2 = 2265.05
alb = (sca1+sca2)/(ext1+ext2)
print(alb)

0.6412249652459371
0.6743475507144898


In [14]:
#WD5.5 r then g
#g:
weightg(0.610595, 0.49249, 2863.63, 451.709)
weightg(0.6297, 0.5407, 3645.40, 492.108)
#albedo:
# weightg(0.890591, 0.370600, 2863.63, 451.709)
# weightg(0.883528, 0.353876, 3645.40, 492.108)
# weightg(1.24,0.02998,133.197, 43.7558)
# weightg(1.183,0.0297,174.598, 47.5795)

sca1 = 2863.63
sca2 = 451.709
ext1 = 3215.43
ext2 = 1218.86
alb = (sca1+sca2)/(ext1+ext2)
print(alb)

sca1 = 3645.40
sca2 = 492.108
ext1 = 4125.96
ext2 = 1390.62
alb = (sca1+sca2)/(ext1+ext2)
print(alb)

0.5945034053108897
0.6191144949085295
0.7476594900198228
0.7500132328362862


In [ ]:
#astrodust r then g
g = 0.606206, albedo = 0.759765, sca = 2351.05
g = 0.634559, albedo = 0.750455, sca = 2877.09

#second file ad
g = 0.64629, albedo = 0.771742, sca = 1894.7
g = 0.68211, albedo = 0.773809, sca = 2513.06